<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/PP/outlier_train_per_95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [3]:
# data
path = Path("/content/drive/MyDrive/GSDC")
test_base = pd.read_csv(path / "baseline_locations_test.csv")
sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv", usecols=cols)
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

In [8]:
def outlier_train_per_95(input_df):
    output_df = input_df

    def calc_haversine(lat1, lon1, lat2, lon2):
        RADIUS = 6_367_000
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        dist = 2 * RADIUS * np.arcsin(d**0.5)
        return dist
    
    output_df['dist'] = calc_haversine(output_df.latDeg_truth, output_df.lngDeg_truth,
                                        output_df.latDeg_train_base, output_df.lngDeg_train_base)
    
    per_95 = output_df['dist'].mean() + (output_df['dist'].std() * 2)
    ind = output_df[(output_df['dist'] > per_95)][['dist']].index

    for i in ind:
        output_df.loc[i,'latDeg_train_base'] = (output_df.loc[i-1,'latDeg_train_base'] + output_df.loc[i+1,'latDeg_train_base'])/2
        output_df.loc[i,'lngDeg_train_base'] = (output_df.loc[i-1,'lngDeg_train_base'] + output_df.loc[i+1,'lngDeg_train_base'])/2

    return output_df

In [9]:
a = outlier_train_per_95(all_df)

In [10]:
a.dist.describe()

count    131342.000000
mean          3.846848
std          30.739767
min           0.001338
25%           1.210976
50%           2.065769
75%           3.560001
max        8340.257976
Name: dist, dtype: float64

In [12]:
train_base.latDeg.describe()

count    131342.000000
mean         37.435206
std           0.082712
min          37.321683
25%          37.371575
50%          37.424360
75%          37.469109
max          37.690836
Name: latDeg, dtype: float64